In [3]:
import cv2
import os
import shutil
import sys

# Function Definitions

In [4]:
def parse_timestamp(ts):
    parts = ts.split(":")
    parts = [float(p) for p in parts]
    if len(parts) == 3:
        h, m, s = parts
    else:
        h, m, s = 0, *parts
    return h*3600 + m*60 + s


#----------------------------------------------
# Load episode and do EDA
#----------------------------------------------
def video_info(video_file):
    ''' 
    do some EDA, extract basic frame features about the episode
    '''
    if not os.path.exists(video_file):
        raise FileNotFoundError(video_file)

    vid = cv2.VideoCapture(video_file)
    if not vid.isOpened():
        raise RuntimeError("Cannot open video")

    fps = vid.get(cv2.CAP_PROP_FPS)
    w   = vid.get(cv2.CAP_PROP_FRAME_WIDTH)
    h   = vid.get(cv2.CAP_PROP_FRAME_HEIGHT)
    n   = vid.get(cv2.CAP_PROP_FRAME_COUNT)
    fourcc = int(vid.get(cv2.CAP_PROP_FOURCC))

    # ok = False
    # for _ in range(5):
    #     ret, frame = vid.read()
    #     if ret and frame is not None:
    #         ok = True
    #         break

    # if not ok:
    #     raise RuntimeError("Unable to decode frames")

    ret, frame = vid.read()
    if not ret or frame is None:
        raise RuntimeError("Cannot decode frames")

    vid.release()

    return {
        "fps": fps,
        "width": int(w),
        "height": int(h),
        "num_frames": int(n),
        "fourcc": fourcc,
        "duration_sec": n / fps if fps else None,
    }

#----------------------------------------------
# Extract frames and their metadata:
# -> what are the indices of frames
# -> whare is the intro part etc.
#----------------------------------------------
def extract_frames(video_file, fps_to_save):
    """
    Save frames from video at fps_to_save.
    Returns: output frames dir and number of frames saved.
    """
    vid = cv2.VideoCapture(video_file)
    if not vid.isOpened():
        raise RuntimeError(f"Cannot open video: {video_file}")

    orig_fps = vid.get(cv2.CAP_PROP_FPS)
    frame_step = max(int(orig_fps // fps_to_save), 1)  # save every Nth frame

    base,_ = os.path.splitext(video_file)
    out_dir = base + "-frames"
    os.makedirs(out_dir, exist_ok=True)

    fid = saved = 0
    while True:
        ret, frame = vid.read()
        if not ret:
            break

        if fid % frame_step == 0:
            cv2.imwrite(os.path.join(out_dir, f"frame{saved}.jpg"), frame)
            saved += 1

        fid += 1

    vid.release()
    print(f"[frames ok] saved={saved} frames in {out_dir}")
    return out_dir, saved

def extract_frames_meta(video_file, intro_timestamp=0, fps_to_save=8):
    """
    Returns: list of (saved_index, filename), frames_dir, video_info
    """
    intro_skip_sec = parse_timestamp(intro_timestamp) if isinstance(intro_timestamp, str) else intro_timestamp

    info = video_info(video_file)
    frames_dir = os.path.splitext(video_file)[0] + "-frames"
    os.makedirs(frames_dir, exist_ok=True)

    frames = []
    for f in os.listdir(frames_dir):
        if f.lower().endswith(".jpg"):
            num = int(f[5:-4])
            frames.append((num, f))

    frames.sort(key=lambda x: x[0])

    # skip intro frames if FPS provided
    if fps_to_save:
        discard_count = int(intro_skip_sec * fps_to_save)
        discard_count = min(discard_count, len(frames))
        frames = frames[discard_count:]

    return frames, frames_dir, info




#----------------------------------------------
# Split frames for modeling
#----------------------------------------------
def split_frames(frames, video_file, split_timestamp, fps_to_save):
    """
    Split frames into train/test based on split_timestamp.
    Uses the sorted frames list; everything before the split → train,
    everything after → test. Does not compute FPS-based indices.
    """
    split_sec = parse_timestamp(split_timestamp) if isinstance(split_timestamp, str) else split_timestamp

    # Compute the frame index corresponding to the split time
    split_index = int(split_sec * fps_to_save)
    split_index = min(split_index, len(frames))  # avoid overflow

    train_frames = frames[:split_index]
    test_frames  = frames[split_index:]

    episode_name = os.path.splitext(os.path.basename(video_file))[0]
    frames_dir = os.path.splitext(video_file)[0] + "-frames"

    train_out = f"../data/processed/video/{episode_name}/train/"
    test_out  = f"../data/processed/video/{episode_name}/test/"
    os.makedirs(train_out, exist_ok=True)
    os.makedirs(test_out, exist_ok=True)

    for _, fname in train_frames:
        shutil.copy(os.path.join(frames_dir, fname), os.path.join(train_out, fname))
    for _, fname in test_frames:
        shutil.copy(os.path.join(frames_dir, fname), os.path.join(test_out, fname))

    print(f"[YoHoo! split done :>] saved frames: train={len(train_frames)}, test={len(test_frames)}")
    return train_frames, test_frames
    


# Preprocessing Episodes

> Train-test split

> **Choice of split:**\
> There are 4 main characters that we need to identify so the split time from a given episode is selected based on the equal (rough idea) no of apperances of the all all character in both splits.

In [5]:
# fps -> acc to processign requirements
FPS_TO_SAVE = 4

# Episode 1
EPISODE_1_PATH = "../data/raw/video/Muppets-02-01-01.avi"
EPISODE_1_SKIP_INTRO_TILL = "00:17"
EPISODE_1_SPLIT_AT = "19:30"

# Episode 2
EPISODE_2_PATH = "../data/raw/video/Muppets-02-04-04.avi"
EPISODE_2_SKIP_INTRO_TILL = "00:00"
EPISODE_2_SPLIT_AT = "00:00"

# Episode 3
EPISODE_3_PATH = "../data/raw/video/Muppets-03-04-03.avi"
EPISODE_3_SKIP_INTRO_TILL = "00:08"
EPISODE_3_SPLIT_AT = "00:00"

In [6]:
# Video info
ep1_info = video_info(EPISODE_1_PATH)
print(ep1_info)

# Extract frames
frames_dir, total_saved = extract_frames(EPISODE_1_PATH, fps_to_save=FPS_TO_SAVE)

# Get metadata and skip intro
frames, _, info = extract_frames_meta(EPISODE_1_PATH, intro_timestamp=EPISODE_1_SKIP_INTRO_TILL, fps_to_save=FPS_TO_SAVE)

# Split train/test
train, test = split_frames(frames, EPISODE_1_PATH, split_timestamp=EPISODE_1_SPLIT_AT, fps_to_save=FPS_TO_SAVE)

{'fps': 25.0, 'width': 720, 'height': 544, 'num_frames': 38682, 'fourcc': 877677894, 'duration_sec': 1547.28}
[frames ok] saved=6447 frames in ../data/raw/video/Muppets-02-01-01-frames
[YoHoo! split done :>] saved frames: train=4680, test=1699
